In [3]:
data_bucket = 'subtest3' #데이터가 저장되어 있는 S3 버킷
subfolder = 'diabetes' #S3버킷의 하위 폴더
train = 'train.csv'
valid = 'valid.csv'
test = 'test.csv'

train = pd.read_csv(f's3://{data_bucket}/{subfolder}/train.csv')
valid = pd.read_csv(f's3://{data_bucket}/{subfolder}/valid.csv')
test = pd.read_csv(f's3://{data_bucket}/{subfolder}/test.csv')

In [4]:
import sys
import pandas as pd
from time import sleep

import boto3  #파이썬으로 AWS 에 접속하는 라이브러리
import sagemaker # sagemaker 라이브러리
import s3fs # S3 버킷 안의 파일을 다루기 위한 모듈

In [6]:
region = boto3.Session().region_name    
smclient = boto3.Session().client('sagemaker')

In [7]:
role = sagemaker.get_execution_role() #Sagemaker 역할 생성
s3 = s3fs.S3FileSystem(anon=False) #S3 버킷과 연결

In [6]:
#하이퍼파라미터 튜닝

In [7]:
tuning_job_config = {
    "ParameterRanges": {
      "CategoricalParameterRanges": [],
      "ContinuousParameterRanges": [
        {
          "MaxValue": "1",
          "MinValue": "0",
          "Name": "eta"
        },
        {
          "MaxValue": "2",
          "MinValue": "0",
          "Name": "alpha"
        },
        {
          "MaxValue": "10",
          "MinValue": "1",
          "Name": "min_child_weight"
        }
      ],
      "IntegerParameterRanges": [
        {
          "MaxValue": "20",
          "MinValue": "3",
          "Name": "max_depth"
        }
      ]
    },
    "ResourceLimits": {
      "MaxNumberOfTrainingJobs": 20,
      "MaxParallelTrainingJobs": 3
    },
    "Strategy": "Bayesian",
    "HyperParameterTuningJobObjective": {
      "MetricName": "validation:auc",
      "Type": "Maximize"
    }
  }

In [8]:
from sagemaker.image_uris import retrieve
training_image = sagemaker.image_uris.retrieve('xgboost', region, '1.0-1')

s3_input_train = 's3://subtest3/diabetes/train.csv'
s3_input_validation ='s3://subtest3/diabetes/valid.csv'

#훈련 알고리즘을 지정하는 컨테이너 이미지
training_job_definition = {
    "AlgorithmSpecification": {
      "TrainingImage": training_image,
      "TrainingInputMode": "File"
    },
    "InputDataConfig": [
      {                  
        "ChannelName": "train",  #train 데이터 입력
        "CompressionType": "None",
        "ContentType": "csv",
        "DataSource": {
          "S3DataSource": {
            "S3DataDistributionType": "FullyReplicated",
            "S3DataType": "S3Prefix",
            "S3Uri": s3_input_train
          }
        }
      },
      {
        "ChannelName": "validation", #valid 데이터 입력
        "CompressionType": "None",
        "ContentType": "csv",
        "DataSource": {
          "S3DataSource": {
            "S3DataDistributionType": "FullyReplicated",
            "S3DataType": "S3Prefix",
            "S3Uri": s3_input_validation
          }
        }
      }
    ],
    "OutputDataConfig": { #output 저장
      "S3OutputPath": "s3://{}/{}/output".format(data_bucket,subfolder)
    },
    "ResourceConfig": {   #훈련 작업에 사용할 인스턴스 유형
      "InstanceCount": 1,
      "InstanceType": "ml.m4.xlarge",
      "VolumeSizeInGB": 10
    },
    "RoleArn": role,    
    "StaticHyperParameters": {
      "eval_metric": "auc",
      "num_round": "100",
      "objective": "binary:logistic",
      "rate_drop": "0.3",
      "tweedie_variance_power": "1.4"
    },
    "StoppingCondition": {
      "MaxRuntimeInSeconds": 43200
    }
}

In [1]:
tuning_job_name = "MyTuningJob3"
smclient.create_hyper_parameter_tuning_job(HyperParameterTuningJobName = tuning_job_name,
                                           HyperParameterTuningJobConfig = tuning_job_config,
                                           TrainingJobDefinition = training_job_definition)

NameError: name 'smclient' is not defined

In [8]:
#실행상태 확인
smclient.describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=tuning_job_name)

{'HyperParameterTuningJobName': 'MyTuningJob3',
 'HyperParameterTuningJobArn': 'arn:aws:sagemaker:ap-northeast-2:689918819853:hyper-parameter-tuning-job/mytuningjob3',
 'HyperParameterTuningJobConfig': {'Strategy': 'Bayesian',
  'HyperParameterTuningJobObjective': {'Type': 'Maximize',
   'MetricName': 'validation:auc'},
  'ResourceLimits': {'MaxNumberOfTrainingJobs': 20,
   'MaxParallelTrainingJobs': 3},
  'ParameterRanges': {'IntegerParameterRanges': [{'Name': 'max_depth',
     'MinValue': '3',
     'MaxValue': '20',
     'ScalingType': 'Auto'}],
   'ContinuousParameterRanges': [{'Name': 'eta',
     'MinValue': '0',
     'MaxValue': '1',
     'ScalingType': 'Auto'},
    {'Name': 'alpha', 'MinValue': '0', 'MaxValue': '2', 'ScalingType': 'Auto'},
    {'Name': 'min_child_weight',
     'MinValue': '1',
     'MaxValue': '10',
     'ScalingType': 'Auto'}],
   'CategoricalParameterRanges': []}},
 'TrainingJobDefinition': {'StaticHyperParameters': {'_tuning_objective_metric': 'validation:auc'

#train the model

In [ ]:
data_bucket = 'subtest3' #데이터가 저장되어 있는 S3 버킷
subfolder = 'diabetes' #S3버킷의 하위 폴더
train = 'train.csv'
valid = 'valid.csv'
test = 'test.csv'

train = pd.read_csv(f's3://{data_bucket}/{subfolder}/train.csv')
valid = pd.read_csv(f's3://{data_bucket}/{subfolder}/valid.csv')
test = pd.read_csv(f's3://{data_bucket}/{subfolder}/test.csv')

In [53]:
#데이터를 csv 형식으로 변경   / encode의 경우 외부와 데이터 통신 시 필요!!
train_data = train.to_csv(None, header=False, index=False).encode()
val_data = valid.to_csv(None, header=False, index=False).encode()
test_data = test.to_csv(None, header=True, index=False).encode()

#csv 파일을 s3에 저장
with s3.open(f'{data_bucket}/{subfolder}/processed/train.csv', 'wb') as f:
    f.write(train_data)

with s3.open(f'{data_bucket}/{subfolder}/processed/val.csv', 'wb') as f:
    f.write(val_data)
    
with s3.open(f'{data_bucket}/{subfolder}/processed/test.csv', 'wb') as f:
    f.write(test_data) 

    


In [10]:
#sagemaker 사용 전 csv 데이터 불러오기    
train_input = sagemaker.TrainingInput(s3_data=f's3://{data_bucket}/{subfolder}/processed/train.csv', content_type='csv')
val_input = sagemaker.TrainingInput(s3_data=f's3://{data_bucket}/{subfolder}/processed/val.csv', content_type='csv')    

#best_tuning 값으로 모델 생성

In [11]:
sess = sagemaker.Session()#sagemaker 세션을 저장하는 변수 생성
#컨테이너 설정
container = sagemaker.image_uris.retrieve(
                'xgboost',
                boto3.Session().region_name,
                'latest')
#모델 생성
estimator = sagemaker.estimator.Estimator(
                        container, 
                        role,
                        instance_count=1, 
                        instance_type='ml.m4.xlarge',
                        output_path=f's3://{data_bucket}/{subfolder}/output',   #학습결과 저장 위치
                        sagemaker_session=sess)

estimator.set_hyperparameters(
                        alpha=0.3456,
                        eta=0.09,
                        max_depth=12,
                        min_child_weight=7.7,
                        objective='binary:logistic',
                        eval_metric='auc',
                        num_round=100,
                        early_stopping_rounds=10,
                        rate_drop=0.3,
                        tweedie_variance_power=1.4)

estimator.fit({'train': train_input, 'validation': val_input})



2021-02-16 11:24:51 Starting - Starting the training job...
2021-02-16 11:25:15 Starting - Launching requested ML instancesProfilerReport-1613474691: InProgress
......
2021-02-16 11:26:16 Starting - Preparing the instances for training......
2021-02-16 11:27:17 Downloading - Downloading input data
2021-02-16 11:27:17 Training - Downloading the training image...
2021-02-16 11:27:50 Uploading - Uploading generated training model
2021-02-16 11:27:50 Completed - Training job completed
Arguments: train
[2021-02-16:11:27:37:INFO] Running standalone xgboost training.
[2021-02-16:11:27:37:INFO] File size need to be processed in the node: 0.04mb. Available memory size in the node: 8427.42mb
[2021-02-16:11:27:37:INFO] Determined delimiter of CSV input is ','
[11:27:37] S3DistributionType set as FullyReplicated
[11:27:37] 429x13 matrix with 5577 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2021-02-16:11:27:37:INFO] Determined delimiter of CSV input is ','
[1

In [8]:
#host the model

In [9]:
#별도의 서버 사용하여 학습된 모델 호스팅(추론 전용 서버)
endpoint_name = 'diabetes'

try:
    sess.delete_endpoint(endpoint_name)
    print('Warning: Existing endpoint deleted to make way for your new endpoint.')
    sleep(30)
except:
    pass

In [10]:
#endpoint 생성
predictor = estimator.deploy(
                initial_instance_count=1,
                instance_type='ml.m4.xlarge', 
                endpoint_name=endpoint_name)

-------------!

In [11]:
#다양한 형식의 데이터를 CSV 형식 문자열로 직렬화
from sagemaker.serializers import CSVSerializer
predictor.serializer = CSVSerializer()

In [12]:
#test the model

In [ ]:
#test dataset에 대한 추론 결과 가져오기
def get_prediction(row):
    prediction = round(float(predictor.predict(row[1:]).decode('utf-8'))) #첫 번째열 제외(target이므로)
    return prediction
#test dataset 불러오기
with s3.open(f'{data_bucket}/{subfolder}/processed/test.csv') as f:
    test_data = pd.read_csv(f)
#test dataset에 대한 추론 결과 생성
test_data['prediction'] = test_data.apply(get_prediction, axis=1)
#test_data[:10]


In [14]:
test_data['prediction']

0      1
1      0
2      1
3      1
4      0
      ..
149    0
150    0
151    1
152    0
153    0
Name: prediction, Length: 154, dtype: int64